In [1]:
import pandas as pd

In [56]:
df = pd.read_excel("cat_data.xlsx")

In [57]:
names = ['streaming','clio','wc_holdings','discard','title','bib_id','oclc_num',
         'format','author','holdings_sup','bib_sup','tag_965','lp','historic_circs',
         'comments','reserve_past_few','location']
df.columns = names

In [193]:
df.head()

,streaming,clio,wc_holdings,discard,title,bib_id,oclc_num,format,author,holdings_sup,bib_sup,tag_965,lp,historic_circs,comments,reserve_past_few,location,clio_y_n
0,n,n,132.0,n,Bright Red [sound recording] / Laurie Anderson.,9062442,227189789.0,MUSICAL SOUND RECORDING,"Anderson, Laurie.",Y,Y,NaN,NaN,NaN,NaN,NaN,bar,
1,NaN,NaN,NaN,NaN,"Chain : farm, factory, and the fate of our foo...",11043454,894893030.0,NONMUSICAL SOUND RECORDING,"Genoways, Ted,",Y,Y,NaN,NaN,NaN,NaN,NaN,bar,
2,n,n,39.0,m,Egmont [sound recording] / Beethoven.,398584,80373612.0,MUSICAL SOUND RECORDING,"Beethoven, Ludwig van, 1770-1827.",Y,Y,NaN,y,NaN,Is this an LP??? We don't even have the capaci...,NaN,bar,
3,y,y,733.0,y,Evolution [sound recording] / the United State...,4108195,44124520.0,MUSICAL SOUND RECORDING,United States Air Force Band.,Y,N,NaN,NaN,NaN,NaN,NaN,bar,
4,n,n,1.0,n,Guido Cantelli [sound recording].,9162461,427008859.0,MUSICAL SOUND RECORDING,NaN,Y,Y,NaN,NaN,NaN,NaN,NaN,bar,


In [59]:
#dropping missing oclc nums, since we won't be able to look those up. might require further filtering
df = df.dropna(subset = ['oclc_num'])
df = df.reset_index(drop = True)

In [337]:
#df.head()

In [336]:
# Endpoints Ref: https://platform.worldcat.org/api-explorer/apis/wcapi

import requests
from bs4 import BeautifulSoup
import json


In [242]:
#Determining if its in the CUL system

#urls for cul system
def create_clio_urls(oclc):
    API_Key = "2TESDqXDn9hHxXsyLgAN05uBuNLb78pfTPwL7cngnYbcDJRRwgFAB82hOdh9Jm86HI7MgGDDvIist666"
    OCLC_Sym = "ZCU"
    base_url = "http://www.worldcat.org/webservices/catalog"
    search_endpoint = f"/content/libraries/{oclc}&oclcsymbol={OCLC_Sym}?wskey={API_Key}"
    url = base_url + search_endpoint
    return url

#urls for number of libraries
def create_lib_urls(oclc):
    API_Key = "2TESDqXDn9hHxXsyLgAN05uBuNLb78pfTPwL7cngnYbcDJRRwgFAB82hOdh9Jm86HI7MgGDDvIist666"
    OCLC_Sym = "ZCU"
    base_url = "http://www.worldcat.org/webservices/catalog"
    search_endpoint = f"/content/libraries/{oclc}&frbrGrouping=off&maximumLibraries=100&format=json?wskey={API_Key}"
    url = base_url + search_endpoint
    return url

# clio_urls = []
# lib_urls = []
# for i in df.oclc_num:
#     clio_urls.append(create_clio_urls(i))
#     lib_urls.append(create_lib_urls(i))
df["clio_y_n"] = ''
df["num_libs"] = ''

In [339]:
#test df for api
df_trun = df.head()
df_trun = df_trun.astype({'oclc_num': 'int32'})

#api call to determine if oclc exists in the clio system and how many libraries have it
for index, row in df_trun.iterrows():
    clio_url = create_clio_urls(row['oclc_num'])
    clio_response = requests.get(clio_url)
    clio_soup = BeautifulSoup(clio_response.content,"lxml")
    if (clio_soup.message == None):
        df_trun.at[index, 'clio_y_n'] = "y"
    else:
        df_trun.at[index, 'clio_y_n'] = "n"
    lib_url = create_lib_urls(row['oclc_num'])
    lib_response = requests.get(lib_url)
    lib_soup = BeautifulSoup(lib_response.content,"html.parser")
    lib_soup_json = json.loads(str(lib_soup))
    if ('totalLibCount' in lib_soup_json):
        df_trun.at[index, 'num_libs'] = lib_soup_json['totalLibCount']
    else:
        df_trun.at[index, 'num_libs'] = ''
    
    


In [335]:
df_trun

,streaming,clio,wc_holdings,discard,title,bib_id,oclc_num,format,author,holdings_sup,bib_sup,tag_965,lp,historic_circs,comments,reserve_past_few,location,clio_y_n,num_libs
0,n,n,132.0,n,Bright Red [sound recording] / Laurie Anderson.,9062442,227189789,MUSICAL SOUND RECORDING,"Anderson, Laurie.",Y,Y,NaN,NaN,NaN,NaN,NaN,bar,n,
1,NaN,NaN,NaN,NaN,"Chain : farm, factory, and the fate of our foo...",11043454,894893030,NONMUSICAL SOUND RECORDING,"Genoways, Ted,",Y,Y,NaN,NaN,NaN,NaN,NaN,bar,n,32
2,n,n,39.0,m,Egmont [sound recording] / Beethoven.,398584,80373612,MUSICAL SOUND RECORDING,"Beethoven, Ludwig van, 1770-1827.",Y,Y,NaN,y,NaN,Is this an LP??? We don't even have the capaci...,NaN,bar,n,8
3,y,y,733.0,y,Evolution [sound recording] / the United State...,4108195,44124520,MUSICAL SOUND RECORDING,United States Air Force Band.,Y,N,NaN,NaN,NaN,NaN,NaN,bar,y,707
4,n,n,1.0,n,Guido Cantelli [sound recording].,9162461,427008859,MUSICAL SOUND RECORDING,NaN,Y,Y,NaN,NaN,NaN,NaN,NaN,bar,y,1
